# Metric Analysis for ECON model

In [1]:
%reload_ext autoreload
%autoreload 2

# from common.metrics.gradient import Gradient
import torch
import torchinfo
import pytorch_lightning as pl 
import os
import sys

# import modules from ECON model
module_path = os.path.abspath(os.path.join('../../workspace/models/econ/code/')) # or the path to your source code
sys.path.insert(0, module_path)
from q_autoencoder import AutoEncoder
from autoencoder_datamodule import AutoEncoderDataModule

# import modules from common metrics
module_path = os.path.abspath(os.path.join('../../workspace/common/metrics/')) # or the path to your source code
sys.path.insert(0, module_path)
from gradient import Gradient
from hessian import Hessian



/opt/conda/envs/loss_landscape/lib/python3.8/site-packages/torchvision/io/image.py:11: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


Select the desired ECON model:

In [2]:
base_path = "/loss_landscape/checkpoint/different_knobs_subset_10/"
batch_size = 1024
learning_rate = 0.1
precision = 8
size = 'small'

In [3]:
def load_model(path, batch_size, learning_rate, precision, size):
    model_path = path + f'bs{batch_size}_lr{learning_rate}/ECON_{precision}b/{size}/net_1_best.pkl'
    model = AutoEncoder(
        quantize=(precision < 32),
        precision=[
            precision,
            precision,
            precision+3
        ],
        learning_rate=learning_rate,
        econ_type=size
    )
    model(torch.randn((1, 1, 8, 8)))  # Update tensor shapes 
    print(model_path)
    model_param = torch.load(model_path)
    model.load_state_dict(model_param['state_dict'])
    return model


model = load_model(base_path, batch_size, learning_rate, precision, size)
torchinfo.summary(model, input_size=(1, 1, 8, 8))  # (B, C, H, W)            

/loss_landscape/checkpoint/different_knobs_subset_10/bs1024_lr0.1/ECON_8b/small/net_1_best.pkl


Layer (type:depth-idx)                   Output Shape              Param #
AutoEncoder                              [1, 1, 8, 8]              --
├─QuantizedEncoder: 1-1                  [1, 16]                   --
│    └─QuantAct: 2-1                     [1, 1, 8, 8]              --
│    └─QuantConv2d: 2-2                  [1, 1, 4, 4]              20
│    └─ReLU: 2-3                         [1, 1, 4, 4]              --
│    └─QuantAct: 2-4                     [1, 1, 4, 4]              --
│    └─Flatten: 2-5                      [1, 16]                   --
│    └─QuantLinear: 2-6                  [1, 16]                   272
│    └─ReLU: 2-7                         [1, 16]                   --
├─Sequential: 1-2                        [1, 1, 8, 8]              --
│    └─Linear: 2-8                       [1, 128]                  2,176
│    └─ReLU: 2-9                         [1, 128]                  --
│    └─Unflatten: 2-10                   [1, 8, 4, 4]              --
│    └─Conv

Setup the data loader:

In [4]:
data_path = '../../data/ECON/Elegun'
processed_file = 'nELinks5.npy'

def get_data_module(batch_size):
    data_module = AutoEncoderDataModule(
        data_dir=data_path,
        data_file=os.path.join(data_path, processed_file),
        batch_size=batch_size,
        num_workers=4
        )
    # checek if we have processed the data
    if not os.path.exists(os.path.join(data_path, processed_file)):
        print('Processing the data...')
        data_module.process_data(save=True)

    data_module.setup(0)
    return data_module

data_module = get_data_module(batch_size)

Loaded shaped data shape: (1740950, 1, 8, 8)
Loaded shaped data datatype: float32


Model performances:

In [5]:
trainer = pl.Trainer(
        max_epochs=5,
        accelerator="auto",
        devices=1
    )

def compute_EMD(size, precision, bs, lr):
    # load the model 
    model = load_model(base_path, bs, lr, precision, size)
    # get the data loader
    data_module = get_data_module(bs)
    _, val_sum = data_module.get_val_max_and_sum()
    model.set_val_sum(val_sum)
    data_module.setup("test")
    avg_emd = trainer.test(model, dataloaders=data_module.test_dataloader(), verbose=False)
    print(f'BATCH SIZE: {bs} - LEARNING RATE {lr} - SIZE {size} - PRECISION {precision}')
    print("AVG EMD: ", avg_emd)
    

# compute_EMD('small', 8, batch_size, learning_rate)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


## Gradient computation

In [6]:
%reload_ext autoreload
%autoreload 2

_, val_loader = data_module.dataloaders()

metric = Hessian(
    model=model,
    data_loader=val_loader,
    loss=model.loss
)

results = metric.compute()
results

Computing the hessian metrics...


/opt/conda/envs/loss_landscape/lib/python3.8/site-packages/torch/autograd/__init__.py:200: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ../torch/csrc/autograd/engine.cpp:1151.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


TypeError: is_same_size(): argument 'input' (position 1) must be Tensor, not float

In [ ]:
metric.save_on_file(path=".")

Storing the result...
Complete


In [ ]:
result = metric.load_from_file(path=".")

result

Loading the result...
Loading complete


[[9.405928620358536e-29,
  4.6703023188540395e-27,
  4.6851814386797704e-27,
  4.7295362097852235e-27,
  4.790515246400486e-27,
  4.791632662208541e-27,
  4.791633053507101e-27,
  4.798341569110351e-27,
  4.9545817441657706e-27,
  5.33874255649012e-27,
  5.338748871057693e-27,
  5.338748871057693e-27,
  5.339829117029079e-27,
  5.366688451395128e-27,
  5.366699130056009e-27,
  5.366699130056009e-27,
  5.366702272293468e-27,
  5.3733121425737065e-27,
  6.769661582397597e-27,
  6.769801184764978e-27,
  6.769801184764978e-27,
  6.769801184764978e-27,
  6.769801184811997e-27,
  6.830256990365918e-27,
  1.4025320705321364e-26,
  1.4025320705321364e-26,
  1.4025320705321364e-26,
  1.4025320705321364e-26,
  1.4026093867170392e-26,
  1.4455630805950296e-26,
  1.4455687111377384e-26,
  1.4455687111377384e-26,
  1.4455687111377384e-26,
  1.4455687111377384e-26,
  1.4562110649569165e-26,
  1.478373975650882e-26,
  1.478373975650882e-26,
  1.478373975650882e-26,
  1.478373975650882e-26,
  1.478373